# 04 — Infer Videos → PGN (Submission)

Use board warp + per‑cell CNN + rule engine to output PGN per video.

- Reads videos from `data/public/videos/*.mp4` (local) or a Kaggle input path you specify.
- Loads model from `models/cell_cnn.h5`.
- Writes `submissions/submission.csv` with `row_id,output`.


In [ ]:
# %%capture
# !pip install --quiet opencv-python python-chess tqdm


In [25]:
# =========================
# 04 – Setup & Bootstrap (FIXED, MERGED CFG)
# =========================

# --- stdlib / third-party ---
import os, sys, json, csv
from pathlib import Path
from collections import deque
import numpy as np
import cv2
from tqdm import tqdm

# --- TF / Keras ---
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input  # type: ignore

# --- env: local vs Kaggle ---
ON_KAGGLE = Path("/kaggle").exists()
ROOT = Path("/kaggle/working") if ON_KAGGLE else Path("..").resolve()

# ให้ import โค้ดจากโปรเจกต์ได้ (ทั้งแบบแพ็กเกจและแบบ src/)
sys.path.insert(0, str(ROOT / "src"))
sys.path.insert(0, str(ROOT))

# --- project modules ---
from Chess_Detection_Competition.utils import load_config
from Chess_Detection_Competition.model import load_model as load_cell_model
from Chess_Detection_Competition.pgn import diff_to_move, san_list_to_pgn, labels_to_board

# ---------- เลือกแหล่ง warp/split + DEFAULT_CFG และสร้าง CFG_FOR_BOARD ----------
# พยายามใช้ improved_board ก่อน ถ้าไม่มีจะ fallback ไป board.py
try:
    from Chess_Detection_Competition.improved_board import (
        warp_board_v2 as warp_board,
        split_grid_v2 as split_grid,
        DEFAULT_CFG as BOARD_DEFAULT_CFG,
    )
    print("[INFO] Using improved_board.py (warp_board_v2, split_grid_v2)")
except ImportError:
    from Chess_Detection_Competition.board import (
        warp_board,
        split_grid,
        DEFAULT_CFG as BOARD_DEFAULT_CFG,
    )
    print("[WARN] improved_board.py not found, using board.py")

def build_board_cfg(user_board: dict | None):
    """รวม default จากไฟล์ + override จาก YAML → คืน {"board": merged}"""
    merged = BOARD_DEFAULT_CFG["board"].copy()
    if user_board:
        merged.update(user_board)
    return {"board": merged}

# --- paths ---
VIDEOS_DIR   = ROOT / "data/public/videos"   # input videos
MODEL_PATH   = ROOT / "models/cell_cnn.h5"   # trained model (.h5)
CLASSES_JSON = ROOT / "models/classes.json"  # class order saved at training time
TRAIN_DIR    = ROOT / "data/final/train"     # fallback only (avoid if classes.json exists)
OUT_DIR      = ROOT / "submissions"
OUT_DIR.mkdir(parents=True, exist_ok=True)
SUBMIT_CSV   = OUT_DIR / "submission.csv"

# --- load config (board/cell/inference params) ---
cfg = load_config()  # configs/parameters.yaml

CFG_FOR_BOARD = build_board_cfg(cfg.get("board", {}))  # <<< ใช้อันนี้กับ warp_board ทุกที่
print("CFG_FOR_BOARD =", CFG_FOR_BOARD["board"])       # ต้องเห็น refine_* ครบ

IMG_SIZE    = int(cfg["cells"]["img_size"])
SMOOTH_K    = int(cfg["inference"]["smooth_k"])
SAMPLE_STEP = int(cfg["inference"]["sample_step"])
cfg_source  = "configs/parameters.yaml"

# --- load class order (MUST match training) ---
if CLASSES_JSON.exists():
    CLASSES = json.loads(CLASSES_JSON.read_text(encoding="utf-8"))
    print("Loaded classes from classes.json:", CLASSES)
else:
    subdirs = [p.name for p in TRAIN_DIR.iterdir() if p.is_dir()] if TRAIN_DIR.exists() else []
    CLASSES = sorted(subdirs)
    print("[WARN] classes.json missing. Inferred classes:", CLASSES)

# --- load trained cell-CNN ---
model = load_cell_model(str(MODEL_PATH))
try:
    num_out = model.output_shape[-1]
    assert num_out == len(CLASSES), (
        f"Model output ({num_out}) != num classes ({len(CLASSES)}) "
        f"→ ใช้ classes.json ที่ได้จากรันเทรนเดียวกัน"
    )
    print(f"OK: model outputs {num_out} classes.")
except Exception as e:
    print("[WARN] cannot verify model output vs classes:", e)

# --- debug helpers ---
def _dbg(s): print("[DBG]", s)

def _mask_changes(a, b):
    return np.array([[a[r][c] != b[r][c] for c in range(8)] for r in range(8)], dtype=bool)

def _diff_preview(prev_labels, now_labels, mask=None, max_list=8):
    rows, coords = [], []
    for r in range(8):
        marks = []
        for c in range(8):
            changed = (prev_labels[r][c] != now_labels[r][c])
            if mask is not None:
                changed = changed and bool(mask[r, c])
            marks.append("X" if changed else ".")
        rows.append(" ".join(marks))
    for s in rows: _dbg(s)
    if coords:
        _dbg("changes (r,c: prev→now):")
        for (r,c,a,b) in coords[:max_list]:
            _dbg(f"  ({r},{c}): {a} → {b}")
        if len(coords) > max_list:
            _dbg(f"  ... (+{len(coords)-max_list} more)")

# --- Homography bootstrap ---
FORCE_H0 = None
FORCE_WARP_SIZE = None  # (W,H)

def scan_for_H0(video_path: Path, max_scan=180, step=3):
    """
    หา H0 จากเฟรมแรก ๆ ที่ detect board ได้ชัดเจน
    """
    cap = cv2.VideoCapture(str(video_path))
    H0, wh = None, None
    f = 0

    while f < max_scan:
        ok = cap.grab()
        if not ok:
            break
        f += 1
        if f % step:
            continue
        ok, frame = cap.retrieve()
        if not ok or frame is None:
            continue
        try:
            warped, aux = warp_board(frame, CFG_FOR_BOARD)  # <<< ใช้ CFG_FOR_BOARD
            if warped is not None and getattr(warped, "size", 0) > 0:
                if isinstance(aux, dict) and "H" in aux:
                    H0 = aux["H"]
                elif isinstance(aux, np.ndarray):
                    H0 = aux
                if H0 is not None:
                    wh = (warped.shape[1], warped.shape[0])
                    print(f"[HBOOT] Found H at frame {f}: size={wh}")
                    break
        except Exception:
            continue

    cap.release()
    if H0 is None:
        print(f"[HBOOT] ⚠️ No H found in first ~{max_scan} frames")
    return H0, wh

def run_video_with_homography(video_path: Path):
    global FORCE_H0, FORCE_WARP_SIZE
    FORCE_H0, FORCE_WARP_SIZE = scan_for_H0(video_path, max_scan=180, step=3)
    if FORCE_H0 is None:
        print("[HBOOT] ❌ Proceeding WITHOUT fixed H (results will be VERY noisy)")
    else:
        print("[HBOOT] ✅ Using fixed H for all frames")
    pgn = decode_video_to_pgn(video_path)  # defined in Cell3
    FORCE_H0, FORCE_WARP_SIZE = None, None
    return pgn

# --- environment summary ---
video_files = sorted(VIDEOS_DIR.glob("*.mp4"))
print("=== Environment ===")
print("Kaggle     :", ON_KAGGLE)
print("ROOT       :", ROOT)
print("Config     :", cfg_source)
print("Model path :", MODEL_PATH, "exists?", MODEL_PATH.exists())
print("IMG_SIZE   :", IMG_SIZE)
print("Videos dir :", VIDEOS_DIR, "| found:", len(video_files))


[WARN] improved_board.py not found, using board.py
CFG_FOR_BOARD = {'warp_size': 800, 'canny_low': 60, 'canny_high': 180, 'hough_threshold': 120, 'min_line_length': 120, 'max_line_gap': 10, 'refine_dx': 3, 'refine_dy': 3, 'refine_rot_deg': 1.0, 'refine_scale': 0.01}
Loaded classes from classes.json: ['BB', 'BK', 'BN', 'BP', 'BQ', 'BR', 'Empty', 'WB', 'WK', 'WN', 'WP', 'WQ', 'WR']


OK: model outputs 13 classes.
=== Environment ===
Kaggle     : False
ROOT       : C:\Users\worap\Downloads\image_processing_term_orject\Chess_Detection_Competition
Config     : configs/parameters.yaml
Model path : C:\Users\worap\Downloads\image_processing_term_orject\Chess_Detection_Competition\models\cell_cnn.h5 exists? True
IMG_SIZE   : 96
Videos dir : C:\Users\worap\Downloads\image_processing_term_orject\Chess_Detection_Competition\data\public\videos | found: 5


In [26]:
# =========================
# Inference helpers & tunables
# =========================
import chess
from copy import deepcopy

## ---- Tunables ----
# TAU = 0.45                   # thresh ความมั่นใจต่อ cell
# SMOOTH_K = max(9, int(SMOOTH_K))  # moving average per-cell
# SETTLE = 15                  # warm-up frames
# SAMPLE_STEP = 1              # อ่านทุกเฟรม
# MIN_CH, MAX_CH = 2, 8        # เกณฑ์จำนวนช่องที่ควรเปลี่ยนใน 1 move
# PENDING_HORIZON = 2
# ENFORCE_LEGAL = True         # ใช้กฎหมากรุกจริง
# REQUIRE_STABLE_FRAMES = 2    # label ใหม่ต้องคงอยู่กี่เฟรม ถึงจะยอม “เปลี่ยน”
# ---- Tunables (อัปเดต) ----
TAU = 0.55                 # เดิม 0.45
SMOOTH_K = max(11, SMOOTH_K)
SETTLE = 20
SAMPLE_STEP = 1

MIN_CH, MAX_CH = 2, 6      # เดิม 2..8
PENDING_HORIZON = 6
REQUIRE_STABLE_FRAMES = 2
ENFORCE_LEGAL = True       # ไว้เปิดก่อน ถ้าไม่ได้ค่อยปิดทดสอบ


def predict_labels8x8_with_conf(warped_bgr, buffers):
    cells = split_grid(warped_bgr, IMG_SIZE)
    X = []
    for _, patch in cells:
        rgb = cv2.cvtColor(patch, cv2.COLOR_BGR2RGB).astype(np.float32)
        X.append(preprocess_input(rgb))
    X = np.asarray(X, dtype=np.float32)
    probs = model.predict(X, verbose=0)  # (64, C)

    labels = [[None]*8 for _ in range(8)]
    confs  = np.zeros((8, 8), dtype=np.float32)
    k = 0
    for r in range(8):
        for c in range(8):
            buffers[r][c].append(probs[k])
            avg = np.mean(np.stack(buffers[r][c], axis=0), axis=0)
            idx = int(np.argmax(avg))
            labels[r][c] = CLASSES[idx]
            confs[r, c]  = float(avg[idx])
            k += 1
    return labels, confs

# ---- legal-move resolver ----
def rc_to_square(r, c):
    file = c
    rank = 7 - r
    return chess.square(file, rank)

def find_from_to_pair(prev_labels, now_labels, eff_mask):
    srcs, dsts = [], []
    for r in range(8):
        for c in range(8):
            if not eff_mask[r, c]:
                continue
            a, b = prev_labels[r][c], now_labels[r][c]
            if a != "Empty" and b == "Empty":
                srcs.append((r, c, a))
            if a == "Empty" and b != "Empty":
                dsts.append((r, c, b))
    if not srcs or not dsts:
        return None
    best, best_d = None, 1e9
    for (rs, cs, pa) in srcs:
        for (rd, cd, pb) in dsts:
            d = abs(rs - rd) + abs(cs - cd)
            if d < best_d:
                best_d = d
                best = ((rs, cs, pa), (rd, cd, pb))
    return best

def make_move_candidates(rs, cs, rd, cd, try_promos=("Q","R","B","N")):
    from_sq = rc_to_square(rs, cs)
    to_sq   = rc_to_square(rd, cd)
    cands = [chess.Move(from_sq, to_sq)]
    for sym in try_promos:
        promo = {"Q": chess.QUEEN, "R": chess.ROOK, "B": chess.BISHOP, "N": chess.KNIGHT}[sym]
        cands.append(chess.Move(from_sq, to_sq, promotion=promo))
    return cands

def resolve_move_by_legality(prev_labels, now_labels, eff_mask, try_promos=("Q","R","B","N")):
    b_prev = labels_to_board(prev_labels)
    pair = find_from_to_pair(prev_labels, now_labels, eff_mask)
    if pair is None:
        return None
    (rs, cs, _), (rd, cd, _) = pair
    candidates = make_move_candidates(rs, cs, rd, cd, try_promos=try_promos)
    legal = set(b_prev.legal_moves)
    for mv in candidates:
        if mv in legal:
            return mv
    return None


In [27]:
# =========================
# Orientation helpers (ใส่ไว้ก่อน decode_video_to_pgn)
# =========================
import numpy as np

def rotate_labels(labels, k):  # k = 0,1,2,3 (หมุน 90° ตามเข็ม)
    x = np.array(labels)
    for _ in range(k):
        x = np.rot90(x, k=-1)
    return x.tolist()

def orientation_score(lbls):
    # ให้คะแนนสูงเมื่อฝั่งล่างเป็น “ขาวเยอะ” และฝั่งบนเป็น “ดำเยอะ”
    W = {"WP","WR","WN","WB","WQ","WK"}
    B = {"BP","BR","BN","BB","BQ","BK"}
    bottom = sum(lbl in W for r in range(4,8) for lbl in lbls[r])
    top    = sum(lbl in B for r in range(0,4) for lbl in lbls[r])
    return bottom + top

def _dump_changes(prev_labels, now_labels, eff_mask, now_confs, tau):
    """
    พิมพ์รายการ cell ที่เปลี่ยน (ตาม eff_mask) พร้อม confidence ของเฟรมปัจจุบัน
    * prev_labels, now_labels: list[list[str]] 8x8
    * eff_mask: np.ndarray(bool) 8x8 – จุดที่นับว่า "เปลี่ยนจริง" (ตาม gating)
    * now_confs: np.ndarray(float32) 8x8 – ความมั่นใจของโมเดลในเฟรมนี้
    * tau: float – เกณฑ์ sticky/เชื่อถือ (เช่น 0.55–0.60)
    """
    coords = [(r, c) for r in range(8) for c in range(8) if eff_mask[r, c]]
    _dbg(f"[CHG] n={len(coords)} -> {coords[:8]}{' ...' if len(coords) > 8 else ''}")
    for (r, c) in coords:
        a = prev_labels[r][c]
        b = now_labels[r][c]
        conf = float(now_confs[r, c])
        star = " *" if conf >= tau else ""
        _dbg(f"  ({r},{c}): {a} -> {b} | conf={conf:.2f}{star}")



In [28]:
# =========================
# Main inference: decode_video_to_pgn (มี orientation + ใช้ CFG_FOR_BOARD)
# =========================
from copy import deepcopy
import chess

def decode_video_to_pgn(video_path: Path) -> str:
    cap = cv2.VideoCapture(str(video_path))
    ok, frame0 = cap.read()
    if not ok or frame0 is None:
        _dbg(f"read fail: {video_path.name}")
        cap.release(); return ""

    # --- warp frame0 ---
    if FORCE_H0 is not None and FORCE_WARP_SIZE is not None:
        warped0 = cv2.warpPerspective(frame0, FORCE_H0, FORCE_WARP_SIZE)
        aux0 = {"H": FORCE_H0}
        _dbg("[HUSE] using FORCE_H0 on frame0")
    else:
        # ใช้พารามิเตอร์จาก YAML ที่โหลดเป็น CFG_FOR_BOARD (dict ที่มี key "board": {...})
        warped0, aux0 = warp_board(frame0, CFG_FOR_BOARD)

    if warped0 is None or getattr(warped0, "size", 0) == 0:
        _dbg(f"warp fail: {video_path.name}")
        cap.release(); return ""

    H0 = None
    if isinstance(aux0, dict) and "H" in aux0:
        H0 = aux0["H"]
    elif isinstance(aux0, np.ndarray):
        H0 = aux0

    _dbg(f"frame0 shape={frame0.shape}, warped shape={warped0.shape}, have H? {H0 is not None}")

    # --- buffers / state ---
    buffers = [[deque(maxlen=max(9, int(SMOOTH_K))) for _ in range(8)] for __ in range(8)]
    stable  = [[None]*8 for _ in range(8)]
    steady  = [[0]*8 for _ in range(8)]

    # --- predict เฟรมแรก ---
    prev_labels, prev_confs = predict_labels8x8_with_conf(warped0, buffers)

    # --- เลือก orientation จากเฟรมแรก แล้วใช้ตลอดทั้งวิดีโอ ---
    cands = [(k, rotate_labels(prev_labels, k)) for k in range(4)]
    ORIENT_K, prev_labels = max(cands, key=lambda t: orientation_score(t[1]))
    _dbg(f"[ORIENT] choose k={ORIENT_K}")

    # init stable/steady
    for r in range(8):
        for c in range(8):
            stable[r][c] = prev_labels[r][c]
            steady[r][c] = 2  # ให้เริ่มที่ค่านี้เพื่อ “นับว่าคงตัวแล้ว” เบื้องต้น
    _dbg(f"first labels sample: {prev_labels[0][:4]} ...")

    # --- warm-up (เติมบัฟเฟอร์) ---
    settle_cfg = int(cfg.get("inference", {}).get("settle", 3))
    for _ in range(settle_cfg):
        ok, fr = cap.read()
        if not ok: break
        if FORCE_H0 is not None and FORCE_WARP_SIZE is not None:
            warped = cv2.warpPerspective(fr, FORCE_H0, FORCE_WARP_SIZE)
        else:
            if H0 is not None:
                warped = cv2.warpPerspective(fr, H0, (warped0.shape[1], warped0.shape[0]))
            else:
                warped, _ = warp_board(fr, CFG_FOR_BOARD)

        if warped is None or getattr(warped, "size", 0) == 0:
            continue
        now_labels_raw, _ = predict_labels8x8_with_conf(warped, buffers)
        # เติมบัฟเฟอร์โดยไม่อัปเดตสถานะหลัก
        _ = rotate_labels(now_labels_raw, ORIENT_K)

    # --- tunables (ค่าเริ่มต้น; ถ้าตั้งตัวแปรพวกนี้ไว้ก่อนหน้าแล้ว สามารถคอมเมนต์บล็อกนี้ได้) ---
    TAU = float(os.environ.get("TAU", "0.60"))     # ความเชื่อมั่นขั้นต่ำก่อนยอมรับ cell
    SAMPLE_STEP_LOC = max(1, int(SAMPLE_STEP))     # เว้นเฟรมย่อย
    REQUIRE_STABLE_FRAMES = 3                      # ต้องคงตัวกี่รอบ
    MIN_CH, MAX_CH = 2, 6                          # จำนวน cell เปลี่ยนที่ยอมรับว่า "น่าจะเป็น 1 move"
    ENFORCE_LEGAL = True                           # บังคับตรวจ legality ด้วยกติกาหมากรุก
    PENDING_HORIZON = 4                            # รวม diff 2–3 เฟรมข้างเคียงเมื่อยังไม่ commit

    # --- main loop ---
    sans = []
    frame_id = settle_cfg
    pending = None

    while True:
        ok, frame = cap.read()
        if not ok: break
        frame_id += 1
        if frame_id % SAMPLE_STEP_LOC:
            continue

        # warp ทุกเฟรม
        if FORCE_H0 is not None and FORCE_WARP_SIZE is not None:
            warped = cv2.warpPerspective(frame, FORCE_H0, FORCE_WARP_SIZE)
        else:
            if H0 is not None:
                warped = cv2.warpPerspective(frame, H0, (warped0.shape[1], warped0.shape[0]))
            else:
                warped, _ = warp_board(frame, CFG_FOR_BOARD)

        if warped is None or getattr(warped, "size", 0) == 0:
            _dbg(f"warp fail midstream @ f{frame_id}")
            continue

        # พยากรณ์ + หมุนตาม ORIENT_K
        now_labels_raw, now_confs = predict_labels8x8_with_conf(warped, buffers)
        now_labels_raw = rotate_labels(now_labels_raw, ORIENT_K)

        # sticky โดย TAU
        sticky = [[now_labels_raw[r][c] if now_confs[r, c] >= TAU else stable[r][c]
                   for c in range(8)] for r in range(8)]

        # per-cell stability
        for r in range(8):
            for c in range(8):
                if sticky[r][c] == stable[r][c]:
                    steady[r][c] = min(steady[r][c] + 1, REQUIRE_STABLE_FRAMES)
                else:
                    steady[r][c] = 1
                if steady[r][c] >= REQUIRE_STABLE_FRAMES:
                    stable[r][c] = sticky[r][c]

        # diff + gating
        conf_or   = (prev_confs >= TAU) | (now_confs >= TAU)
        diff_raw  = _mask_changes(stable, sticky)
        eff_mask  = conf_or & diff_raw
        n_changes = int(eff_mask.sum())
        if frame_id % (5 * max(1, SAMPLE_STEP_LOC)) == 0:
            _dbg(f"f{frame_id}: changes_eff={n_changes}")

        committed = False
        if MIN_CH <= n_changes <= MAX_CH:
            mv = diff_to_move(stable, sticky)
            if ENFORCE_LEGAL and mv is None:
                try_promos = tuple(cfg.get("pgn", {}).get("try_promotions", ["Q","R","B","N"]))
                mv = resolve_move_by_legality(stable, sticky, eff_mask, try_promos=try_promos)

            if mv is not None:
                b_prev = labels_to_board(stable)
                try:
                    san = b_prev.san(mv)
                except Exception as e:
                    _dbg(f"SAN fail @ f{frame_id}: {e}")
                    _diff_preview(stable, sticky, eff_mask)
                else:
                    sans.append(san)
                    _dbg(f"f{frame_id}: SAN={san} (eff={n_changes})")
                    prev_labels = deepcopy(sticky)
                    prev_confs  = now_confs.copy()
                    stable      = deepcopy(sticky)
                    steady      = [[REQUIRE_STABLE_FRAMES]*8 for _ in range(8)]
                    pending     = None
                    committed   = True
            else:
                _dbg(f"f{frame_id}: no {'legal ' if ENFORCE_LEGAL else ''}move matched (eff={n_changes})")
                _diff_preview(stable, sticky, eff_mask)
                _dump_changes(stable, sticky, eff_mask, now_confs, TAU)  # ดูก่อน–หลัง+confidence
                pending = {"frame": frame_id, "labels": deepcopy(stable), "mask": eff_mask.copy()}

        # pending combine 1–2 เฟรมถัดไป
        if not committed and pending is not None and (frame_id - pending["frame"]) <= PENDING_HORIZON:
            combined_mask = pending["mask"] | eff_mask
            combined_changes = int(combined_mask.sum())
            mv2 = diff_to_move(stable, sticky)
            if ENFORCE_LEGAL and mv2 is None:
                try_promos = tuple(cfg.get("pgn", {}).get("try_promotions", ["Q","R","B","N"]))
                mv2 = resolve_move_by_legality(stable, sticky, combined_mask, try_promos=try_promos)
            if mv2 is not None:
                b_prev = labels_to_board(stable)
                try:
                    san = b_prev.san(mv2)
                except Exception as e:
                    _dbg(f"SAN fail (pending) @ f{frame_id}: {e}")
                    _diff_preview(stable, sticky, combined_mask)
                else:
                    sans.append(san)
                    _dbg(f"f{frame_id}: SAN={san} (pending, combined={combined_changes})")
                    prev_labels = deepcopy(sticky)
                    prev_confs  = now_confs.copy()
                    stable      = deepcopy(sticky)
                    steady      = [[REQUIRE_STABLE_FRAMES]*8 for _ in range(8)]
                    pending     = None

        if pending is not None and (frame_id - pending["frame"]) > PENDING_HORIZON:
            pending = None

    cap.release()
    _dbg(f"done. total SAN moves = {len(sans)}")
    if sans: _dbg(f"first SAN moves = {sans[:10]}")
    try:
        return san_list_to_pgn(sans)
    except Exception as e:
        _dbg(f"san_list_to_pgn error: {e}")
        return " ".join(sans)


In [29]:
# =========================
# Visualization & Debug Overlay (NEW)
# =========================
import cv2, numpy as np
from pathlib import Path
from matplotlib import pyplot as plt

def _cell_color(conf, tau):
    return (40,180,40) if conf >= tau else (0,165,255)  # green / orange

def _piece_color(lbl):
    if lbl.startswith("W"): return (255,200,100)  # light blue-ish
    if lbl.startswith("B"): return (180,120,255)  # pink-ish
    return (180,180,180)                           # gray

def annotate_cells(warped_bgr, labels, confs, tau=0.60, xs=None, ys=None):
    img = warped_bgr.copy()
    H, W = img.shape[:2]
    if xs is None: xs = np.linspace(0, W, 9).astype(int)
    if ys is None: ys = np.linspace(0, H, 9).astype(int)
    for x in xs: cv2.line(img, (x,0), (x,H-1), (255,255,255), 1, cv2.LINE_AA)
    for y in ys: cv2.line(img, (0,y), (W-1,y), (255,255,255), 1, cv2.LINE_AA)
    for r in range(8):
        for c in range(8):
            x0, x1 = int(xs[c]), int(xs[c+1])
            y0, y1 = int(ys[r]), int(ys[r+1])
            conf = float(confs[r, c]) if confs is not None else 0.0
            lbl  = labels[r][c]
            cv2.rectangle(img, (x0,y0), (x1,y1), _cell_color(conf, tau), 2, cv2.LINE_AA)
            cv2.putText(img, f"{lbl} {conf:.2f}", (x0+4, y0+18),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, _piece_color(lbl), 1, cv2.LINE_AA)
    return img

def debug_frame_overlay(frame_bgr, tau=0.60, orient_k=None):
    # warp
    if FORCE_H0 is not None and FORCE_WARP_SIZE is not None:
        warped = cv2.warpPerspective(frame_bgr, FORCE_H0, FORCE_WARP_SIZE)
    else:
        warped, _ = warp_board(frame_bgr, CFG_FOR_BOARD)
    if warped is None or getattr(warped, "size", 0) == 0:
        raise RuntimeError("warp fail")

    # predict 8x8
    labels, confs = predict_labels8x8_with_conf(warped, [[deque(maxlen=max(9, int(SMOOTH_K))) for _ in range(8)] for __ in range(8)])

    # orient once
    if orient_k is None:
        cands = [(k, rotate_labels(labels, k)) for k in range(4)]
        orient_k, labels = max(cands, key=lambda t: orientation_score(t[1]))
    else:
        for _ in range(orient_k):
            labels = np.rot90(np.array(labels), k=-1).tolist()
            confs  = np.rot90(confs, k=-1)

    overlay = annotate_cells(warped, labels, confs, tau=tau)
    return overlay, labels, confs, orient_k

def preview_video_with_overlay(video_path: Path, out_path: Path=None,
                               step=3, max_frames=150, tau=0.60):
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened(): raise RuntimeError(f"open fail: {video_path}")

    writer, orient_k, n = None, None, 0
    while True:
        ok = cap.grab()
        if not ok: break
        if (n % step) != 0:
            n += 1; continue
        ok, fr = cap.retrieve()
        if not ok or fr is None: break

        overlay, labels, confs, orient_k = debug_frame_overlay(fr, tau=tau, orient_k=orient_k)

        if out_path:
            if writer is None:
                H,W = overlay.shape[:2]
                writer = cv2.VideoWriter(str(out_path), cv2.VideoWriter_fourcc(*"mp4v"), 20, (W,H))
            writer.write(overlay)
        else:
            plt.figure(figsize=(6,6))
            plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
            plt.axis("off")
            plt.show()

        n += 1
        if n >= max_frames: break

    cap.release()
    if writer is not None: writer.release()
    return True


In [ ]:
# 1) เตรียมของที่ต้องใช้
from collections import deque  # กัน NameError
from pathlib import Path

# 2) เลือกวิดีโอทดสอบ 1 ไฟล์
video_files = sorted(VIDEOS_DIR.glob("*.mp4"))
assert video_files, f"no videos in {VIDEOS_DIR}"
video = video_files[0]
print("Use video:", video.name)

# 3) ล็อก homography ให้เสถียร (แนะนำมากสำหรับ overlay)
FORCE_H0, FORCE_WARP_SIZE = scan_for_H0(video, max_scan=180, step=3)
print("H0 fixed?" , FORCE_H0 is not None)

# 4) แสดง overlay บนโน้ตบุ๊ก (ดูทีละเฟรม)
_ = preview_video_with_overlay(
    video_path=video,
    out_path=None,      # None = แสดงรูปใน notebook, ถ้าอยากบันทึกเป็นไฟล์ให้ตั้งเป็น Path
    step=5,             # ข้ามเฟรมเพื่อวิ่งเร็วขึ้น
    max_frames=100,     # จำนวนเฟรมที่จะพรีวิว
    tau=0.60            # เกณฑ์ความมั่นใจสีกรอบ (เขียว/ส้ม)
)

# 5) (ทางเลือก) บันทึกเป็น mp4 เพื่อเปิดดูภายหลัง
out_mp4 = OUT_DIR / f"annot_{video.stem}.mp4"
_ = preview_video_with_overlay(
    video_path=video,
    out_path=out_mp4,   # เซฟวิดีโอ overlay
    step=3,
    max_frames=300,
    tau=0.60
)
print("Saved overlay video to:", out_mp4)

# 6) (ถ้าจะกลับไป infer ปกติ) รีเซ็ต H0
FORCE_H0, FORCE_WARP_SIZE = None, None


In [17]:
# =========================
# Quick test one video (uses fixed H0)
# =========================
video_files = sorted(VIDEOS_DIR.glob("*.mp4"))
print("videos:", len(video_files))

if video_files:
    p = video_files[0]
    print("TEST:", p.name)
    # ให้เห็น log ชัด ๆ ระหว่างทดสอบ
    SMOOTH_K = 1
    SAMPLE_STEP = 1
    pgn = run_video_with_homography(p)
    print("\nPGN =", pgn if pgn else "[EMPTY]")
else:
    print("[WARN] no videos found")


videos: 5
TEST: 2_Move_rotate_student.mp4
[HBOOT] Found H at frame 3: size=(800, 800)
[HBOOT] ✅ Using fixed H for all frames
[DBG] [HUSE] using FORCE_H0 on frame0
[DBG] frame0 shape=(1920, 1080, 3), warped shape=(800, 800, 3), have H? True
[DBG] [ORIENT] choose k=0
[DBG] first labels sample: ['BR', 'BB', 'BB', 'BK'] ...
[DBG] f5: changes_eff=1
[DBG] f10: changes_eff=2
[DBG] f10: no legal move matched (eff=2)
[DBG] . . . . . . . .
[DBG] . . . . . . X .
[DBG] . . . . . . . .
[DBG] . . . . . . . .
[DBG] . . . . . . . .
[DBG] . . X . . . . .
[DBG] . . . . . . . .
[DBG] . . . . . . . .
[DBG] f11: no legal move matched (eff=3)
[DBG] . . . . . . . .
[DBG] . . . . . . X .
[DBG] . . . . . . . .
[DBG] X . . . . . . .
[DBG] . . . . . . . .
[DBG] . . X . . . . .
[DBG] . . . . . . . .
[DBG] . . . . . . . .
[DBG] f12: no legal move matched (eff=4)
[DBG] . . . . . . . .
[DBG] . . . . . . X .
[DBG] . X . . . . . .
[DBG] X . . . . . . .
[DBG] . . . . . . . .
[DBG] . . X . . . . .
[DBG] . . . . . . . .


KeyboardInterrupt: 

In [24]:
rows = []
# video_files = sorted(VIDEOS_DIR.glob("*.mp4"))
video_files = sorted(VIDEOS_DIR.glob("*.mp4"))[:1]  # ใช้แค่ 1 ไฟล์แรก

if not video_files:
    print(f"[warn] no videos found in: {VIDEOS_DIR}")

for v in tqdm(video_files, desc="Decoding videos"):
    row_id = v.stem
    pgn = decode_video_to_pgn(v)
    rows.append((row_id, pgn))
    print(f"{row_id} -> {pgn}")

with open(SUBMIT_CSV, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow(["row_id", "output"])
    w.writerows(rows)

print("✅ submission saved to:", SUBMIT_CSV.resolve())


Decoding videos:   0%|          | 0/1 [00:00<?, ?it/s]

[DBG] frame0 shape=(1920, 1080, 3), warped shape=(800, 800, 3), have H? True
[DBG] [ORIENT] choose k=0
[DBG] first labels sample: ['WN', 'BN', 'BB', 'BK'] ...
[DBG] f5: changes_eff=0
[DBG] f10: changes_eff=0
[DBG] f13: no legal move matched (eff=2)
[DBG] . X . . . . . .
[DBG] . . . . . . . .
[DBG] . . . . . . . .
[DBG] . . . . . . X .
[DBG] . . . . . . . .
[DBG] . . . . . . . .
[DBG] . . . . . . . .
[DBG] . . . . . . . .
[DBG] [CHG] n=2 -> [(0, 1), (3, 6)]
[DBG]   (0,1): BN -> BB | conf=0.62 *
[DBG]   (3,6): Empty -> WP | conf=0.60 *
[DBG] f14: no legal move matched (eff=2)
[DBG] . X . . . . . .
[DBG] . . . . . . . .
[DBG] . . . . . . . .
[DBG] . . . . . . X .
[DBG] . . . . . . . .
[DBG] . . . . . . . .
[DBG] . . . . . . . .
[DBG] . . . . . . . .
[DBG] [CHG] n=2 -> [(0, 1), (3, 6)]
[DBG]   (0,1): BN -> BB | conf=0.61 *
[DBG]   (3,6): Empty -> WP | conf=0.61 *
[DBG] f15: changes_eff=2
[DBG] f15: no legal move matched (eff=2)
[DBG] . X . . . . . .
[DBG] . . . . . . . .
[DBG] . . . . . . 

Decoding videos: 100%|██████████| 1/1 [11:42<00:00, 702.47s/it]

[DBG] f1578: no legal move matched (eff=3)
[DBG] . . . . . . . .
[DBG] . . . . . X . .
[DBG] . . . . . . . .
[DBG] . . . . . . . .
[DBG] . . . . . . . .
[DBG] . . . . . . . .
[DBG] . X . . . . . .
[DBG] . . X . . . . .
[DBG] [CHG] n=3 -> [(1, 5), (6, 1), (7, 2)]
[DBG]   (1,5): BP -> Empty | conf=0.72 *
[DBG]   (6,1): BP -> Empty | conf=0.76 *
[DBG]   (7,2): WB -> Empty | conf=0.97 *
[DBG] done. total SAN moves = 0
2_Move_rotate_student -> 
✅ submission saved to: C:\Users\worap\Downloads\image_processing_term_orject\Chess_Detection_Competition\submissions\submission.csv
